In [1]:
import os
import dlib
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.patheffects as path_effects


In [2]:
landpath=r'C:\Users\82108\Desktop\dlib_model_and_pred\shape_predictor_68_face_landmarks.dat'
recpath=r'C:\Users\82108\Desktop\dlib_model_and_pred\dlib_face_recognition_resnet_model_v1.dat'

detector=dlib.get_frontal_face_detector()
shaper=dlib.shape_predictor(landpath)
facerec=dlib.face_recognition_model_v1(recpath)

img_path='t.jpeg'
desc_dec=list()


In [3]:

def face_finder(img):
    face_marks=list()
    face_rect=list()
    face_score=list()
    detected,score,_=detector.run(img,1,-0.8)
    if not detected:
        return face_marks,face_rect,face_score

    for i,face in enumerate(detected):
        rect=((face.left(),face.top()),(face.right(),face.bottom()))
        shape=shaper(img,face)

        face_marks.append(shape)
        face_rect.append(rect)
        face_score.append(score[i])
    return face_marks,face_rect,face_score


In [4]:

def face_encode(img,face_marks):
    face_descriptors=list()
    
    for mark in face_marks:
        descriptor=facerec.compute_face_descriptor(img,mark)
        face_descriptors.append(np.array(descriptor))

    return np.array(face_descriptors)

def face_recognition(face_descriptors,img,rects):
    fig,ax=plt.subplots(1,figsize=(20,20))
    ax.imshow(img)
    
    for i,desc in enumerate(face_descriptors):
        # print(rects)
        rect=patches.Rectangle(rects[i][0],
            rects[i][1][1]-rects[i][0][1],
            rects[i][1][0]-rects[i][0][0],
            linewidth=2,edgecolor='w',facecolor='none')
        ax.add_patch(rect)
    # plt.show()


In [5]:
def make_result(score,rects,number):
    fout=open(number+'.txt','w')
    
    for i,s in enumerate(score):
        left=' '+str(rects[i][0][0])
        top=' '+str(rects[i][0][1])
        right=' '+str(rects[i][1][0])
        bottom=' '+str(rects[i][1][1])
        
        line='1 '+str(score[i])+left+top+right+bottom+'\n'
        fout.write(line)
    print('make result')
    fout.close()

In [6]:
def main():
    img_number=0
    img_path='test_img\\'
    while(img_number<319):
        
        number=str('%07d'%img_number)
        print(img_path+number+'.jpg')    
        try:
            
            read_img=cv2.imread(img_path+number+'.jpg')
            print(read_img.shape)
        except:
            print("except")
            break
        img_rgb=cv2.cvtColor(read_img,cv2.COLOR_BGR2RGB)
    
        face_marks,face_rects,face_scores=face_finder(img_rgb)
        face_descriptors=face_encode(img_rgb,face_marks)
        face_recognition(face_descriptors,img_rgb,face_rects)
        make_result(face_scores,face_rects,number)
        img_number+=1
    print("end")
        

In [ ]:
main()